In [1]:
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from copulae1 import *
from KDEs import *
from toolbox import *
import warnings
import itertools

warnings.filterwarnings("ignore")

data = pd.read_csv('../processed_data/btc_future_crix.csv')
data.head()

ecdf_brr = ECDF(data.return_brr)
ecdf_btc = ECDF(data.return_btc)

u = ecdf_brr(data.return_brr)
v = ecdf_btc(data.return_btc)

brr = np.array(data.return_brr)
btc = np.array(data.return_btc)

kde_brr = KDE(data.return_brr, "Gaussian")
kde_btc = KDE(data.return_btc, "Gaussian")

kde_brr = KDE(data.return_brr, "Gaussian", kde_brr.h_brot*3)
kde_btc = KDE(data.return_brr, "Gaussian", kde_btc.h_brot*3)

In [2]:
# Gaussian, t_Copula, Clayton, Frank, Gumbel, Plackett, Gaussian mix Indep
C1  = Gaussian(dict(rho=0.9),       Law_RS=kde_brr, Law_RF=kde_btc) # fix the maringals!
C2  = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=2) 
C2c = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=4) 
C3  = Clayton(dict(theta=0.1),      Law_RS=kde_brr, Law_RF=kde_btc)
C4  = Frank(dict(theta=0.1),        Law_RS=kde_brr, Law_RF=kde_btc)
C5  = Gumbel(dict(theta=3),         Law_RS=kde_brr, Law_RF=kde_btc)
C6  = Plackett(dict(theta=10),      Law_RS=kde_brr, Law_RF=kde_btc)
C7  = Gaussian_Mix_Independent(dict(rho=.5,p=0.7),Law_RS=kde_brr, Law_RF=kde_btc)
Copulae_names = ['Gaussian', 't_Copula', 't_Copula_Capped', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']
Copulae_arr   = [C1,C2,C2c,C3,C4,C5,C6,C7]
Copulae = dict(zip(Copulae_names, Copulae_arr))

In [21]:
m = np.array([stats.kendalltau(u,v)[0]]+[empirical_lambda(u,v,q) for q in q_arr]).reshape((1+len(q_arr)))
m_hat = np.array([C.tau()]+[C._lambda(q) for q in q_arr]).reshape((1+len(q_arr)))
g = m-m_hat
g = g.reshape((-1,1))
g.T.dot(g)[0][0]

0.3496051720530173

In [6]:
q_arr = [0.05,0.1,0.9,0.95]
C=C7
result = C.mm_calibrate(u, v, q_arr)
print(result)
print(C.paras)
print(C.moment_conditions(C.paras, u, v, q_arr))
print(C.paras)

Optimization terminated successfully.
         Current function value: 0.000755
         Iterations: 77
         Function evaluations: 157
[0.94308968 0.78423353]
{'rho': 0.9430896793114437, 'p': 0.7842335278056969}
            Spearman Rho  lambda 0.05  lambda 0.1  lambda 0.9  lambda 0.95
Empirical       0.733723     0.558140    0.635659    0.620155     0.589147
Parametric      0.735470     0.579086    0.621149    0.621149     0.579086
Difference     -0.001747    -0.020946    0.014510   -0.000994     0.010061
{'rho': 0.9430896793114437, 'p': 0.7842335278056969}


In [8]:
result = C.canonical_calibrate(u, v)
print(C.moment_conditions(C.paras, u, v, q_arr))
print(C.paras)
print(result)

Optimization terminated successfully.
         Current function value: -0.499976
         Iterations: 72
         Function evaluations: 153
            Spearman Rho  lambda 0.05  lambda 0.1  lambda 0.9  lambda 0.95
Empirical       0.733723     0.558140    0.635659    0.620155     0.589147
Parametric      0.666157     0.512902    0.558940    0.558940     0.512902
Difference      0.067566     0.045237    0.076719    0.061215     0.076245
{'rho': 0.9275260344030353, 'p': 0.7232928624226377}
[0.92752603 0.72329286]


In [ ]:
C3._lambda(0.1)

In [ ]:
np.array([C.tau()]+[C._lambda(q) for q in q_arr]).reshape((1+len(q_arr)))
# m = np.array([stats.spearmanr(u,v)[0]]+[empirical_lambda(u,v,q) for q in q_arr])

In [ ]:
result = C.canonical_calibrate(u, v)
print(C.moment_conditions(C.paras, u, v, q_arr))
print(C.paras)

In [ ]:
fn_toopt = lambda paras: C.mm_loss(paras, u, v, q_arr)
result = scipy.optimize.fmin(fn_toopt, x0=(0.8),
                             xtol=1e-10, 
                             maxiter=5000,
                             maxfun=400)

In [ ]:
scipy.optimize.fmin(fn_toopt, x0=9)[0]

In [11]:
import sys, os

In [12]:
sys.path

['/Users/francis/Google Drive/00_IRTG1792/copula/Py_notebooks',
 '/Users/francis/opt/anaconda3/lib/python37.zip',
 '/Users/francis/opt/anaconda3/lib/python3.7',
 '/Users/francis/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/francis/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/francis/opt/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/francis/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/francis/.ipython']